In [29]:
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Time series of stock price
## AAPL

In [144]:
# Load AAPL data
df = pd.read_csv(
    "https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv")
df.columns = [col.replace("AAPL.", "") for col in df.columns]

In [146]:
# Create figure
fig = go.Figure()

fig.add_trace(
    go.Scatter(x=list(df.Date), y=list(df.High), name='Stock price')
    )

fig.add_trace(
    go.Candlestick(x=df['Date'],
                open=df['Open'], high=df['High'],
                low=df['Low'], close=df['Close'],
                opacity=0.75, name='Candlestick')
)

# Set title
fig.update_layout(
    title_text="AAPL stock price",
    width=1200,
    height=900,
    autosize=False,
)

# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()

# 3D Surface + HeatMap

## Option price as a function of K and T

In [31]:
# load dataset
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/volcano.csv")

# Create figure
fig = go.Figure()

# Add surface trace
fig.add_trace(go.Heatmap(z=df.values.tolist(), colorscale="Viridis"))

# Update plot sizing
fig.update_layout(
    width=800,
    height=900,
    autosize=False,
    margin=dict(t=100, b=0, l=0, r=0),
)

# Update 3D scene options
fig.update_scenes(
    aspectratio=dict(x=1, y=1, z=0.7),
    aspectmode="manual"
)

# Add drowdowns
button_layer_1_height = 1.08
fig.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(
                    args=["colorscale", "Viridis"],
                    label="Viridis",
                    method="restyle"
                ),
                dict(
                    args=["colorscale", "Cividis"],
                    label="Cividis",
                    method="restyle"
                ),
                dict(
                    args=["colorscale", "Blues"],
                    label="Blues",
                    method="restyle"
                ),
                dict(
                    args=["colorscale", "Greens"],
                    label="Greens",
                    method="restyle"
                ),
            ]),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=button_layer_1_height,
            yanchor="top"
        ),
        dict(
            buttons=list([
                dict(
                    args=["reversescale", False],
                    label="False",
                    method="restyle"
                ),
                dict(
                    args=["reversescale", True],
                    label="True",
                    method="restyle"
                )
            ]),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.37,
            xanchor="left",
            y=button_layer_1_height,
            yanchor="top"
        ),
        dict(
            buttons=list([
                dict(
                    args=[{"contours.showlines": False, "type": "contour"}],
                    label="Hide lines",
                    method="restyle"
                ),
                dict(
                    args=[{"contours.showlines": True, "type": "contour"}],
                    label="Show lines",
                    method="restyle"
                ),
            ]),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.58,
            xanchor="left",
            y=button_layer_1_height,
            yanchor="top"
        ),
    ]
)

fig.update_layout(
    annotations=[
        dict(text="colorscale", x=0, xref="paper", y=1.06, yref="paper",
                             align="left", showarrow=False),
        dict(text="Reverse<br>Colorscale", x=0.25, xref="paper", y=1.07,
                             yref="paper", showarrow=False),
        dict(text="Lines", x=0.54, xref="paper", y=1.06, yref="paper",
                             showarrow=False)
    ])

fig.show()

# Bull spread created using call options

### Auxiliary functions

In [59]:
def getStockPriceVector(start=0, end=100, stepsize=0.01):
    N_steps = int( round((end - start) / stepsize) ) 
    result = np.zeros(N_steps + 1)
    result[0] = start
    current = start
    for i in range(1, len(result)):
        current += stepsize
        result[i] = current
    return result

def bullSpreadPayoff(price, K1, K2):
    if price <= K1:
        return 0.0
    elif price > K1 and price < K2 :
        return price - K1
    else:
        return K2 - K1

In [96]:
def getBullSpreadCallPayoff(K1L, K2S, premium_1, premium_2, stock_price_vector=getPriceVector(0, 10, 0.01)):
    """
    INPUT
    K1L = Strike price of long call
    K2S = Strike price of short call
    premium_1 = Debit paid for long call
    premium_2 = Credit received for short call
    stock_price_vector = Numpy array of stock price

    OUTPUT
    Returns payoff vector
    """
    payoff = np.array( [bullSpreadPayoff(p, K1L, K2S) for p in stock_price_vector] )
    net_premium_paid = premium_1 - premium_2
    payoff = payoff - net_premium_paid
    return payoff

In [111]:
def getLossGainValuesForPlotting(stock_price_vector, payoff):
    loss_values = [v for v in payoff if v < 0.0]
    gain_values = [v for v in payoff if v >= 0.0]
    y_data = [loss_values, gain_values]
    x_data = [stock_price_vector[0:len(loss_values)], stock_price_vector[len(loss_values):]]
    return x_data, y_data

In [167]:
K1 = 25.0
K2 = 32.0
premium_1 = 23.0
premium_2 = 20.0
stock_price_vector = getPriceVector(0,50,0.01)
payoff = getBullSpreadCallPayoff(K1, K2, premium_1, premium_2, stock_price_vector)

In [168]:
x_data, y_data = getLossGainValuesForPlotting(stock_price_vector, payoff)
colors = ['rgb(189,100,100)', 'rgb(49,189,130)']
labels = ['Loss', 'Gain']
line_size = [3,4]
fig = go.Figure()

# Plot data
for i in range(0, 2):
    fig.add_trace(go.Scatter(x=x_data[i], y=y_data[i], mode='lines', fill='tozeroy',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
    ))

# Add annotations
fig.add_trace(go.Scatter(
    x = [K1],
    y = [0],
    mode='markers+text',
    name='Long Call Strike (K1)',
    text=['${}'.format(K1)],
    textposition='top center',
    textfont_size=14,
    marker=dict(color='rgb(100,100,189)')   
))

fig.add_trace(go.Scatter(
    x = [K2],
    y = [0],
    mode='markers+text',
    name='Short Call Strike (K2)',
    text=['${}'.format(K2)],
    textposition='bottom center',
    textfont_size=14,
    marker=dict(color='rgb(189,100,100)')   
))

# Edit layout
fig.update_layout(title='Bull Call Spread',
                   xaxis_title='Stock Price',
                   yaxis_title='Payoff',
                   width=900,
                   height=500)

fig.show()

# Analyze effect of spread with a slider

## Assume only thing that varies is Bull Spread but not premiums

In [192]:
def createBullSpreadTraces(spread_size, midpoint=25):
    K1 = midpoint - spread_size
    K2 = midpoint + spread_size
    premium_1 = K1*0.8
    premium_2 = K2*0.5
    stock_price_vector = getPriceVector(0,2*midpoint,0.01)
    payoff = getBullSpreadCallPayoff(K1, K2, premium_1, premium_2, stock_price_vector)
    x_data, y_data = getLossGainValuesForPlotting(stock_price_vector, payoff)
    colors = ['rgb(189,100,100)', 'rgb(49,189,130)']
    labels = ['Loss', 'Gain']
    line_size = [3,4]
    traces = []

    # Plot data
    for i in range(0, 2):
        traces.append(go.Scatter(x=x_data[i], y=y_data[i], mode='lines', fill='tozeroy',
            name=labels[i],
            line=dict(color=colors[i], width=line_size[i]),
            connectgaps=True,
            ))

    # Add annotations
    traces.append(go.Scatter(
        x = [K1],
        y = [0],
        mode='markers+text',
        name='Long Call Strike (K1)',
        text=['${}'.format(K1)],
        textposition='top center',
        textfont_size=14,
        marker=dict(color='rgb(100,100,189)')   
    ))

    traces.append(go.Scatter(
        x = [K2],
        y = [0],
        mode='markers+text',
        name='Short Call Strike (K2)',
        text=['${}'.format(K2)],
        textposition='bottom center',
        textfont_size=14,
        marker=dict(color='rgb(189,100,100)')   
    ))

    return traces

In [193]:
# Create figure
fig = go.Figure()

# Add traces, one for each slider step
for spread_size in np.arange(1,6,1):
    traces = createBullSpreadTraces(spread_size,25)
    for trace in traces:
        fig.add_trace(trace)

for i in range(len(fig.data)):
    fig.data[0].visible = False

for i in range(4):
    fig.data[i].visible = True

# Create and add slider
steps = []
for i in [0, 4, 8, 12, 16]:
    step = dict(
        method='update', 
        args=[{"visible": [False] * len(fig.data)},
            {"title": "Slider switched to step: " + str(i//4)}],
    )
    step["args"][0]["visible"][i] = True # Toggle i'th trace to "visible"
    step["args"][0]["visible"][i+1] = True
    step["args"][0]["visible"][i+2] = True
    step["args"][0]["visible"][i+3] = True
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Spread: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

fig.show()